# 4. Batch Inference Pipeline

## 4.1. Setup

### 4.1.1. Import Libraries

In [ ]:
# Standard imports
import os
from pathlib import Path
import sys
import json
import time
from datetime import date, datetime, timedelta
from dotenv import load_dotenv

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import numpy as np
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from confluent_kafka import KafkaException
from hsfs.client.exceptions import RestAPIError
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from scipy.spatial.distance import cdist
import traceback
import subprocess
import glob
import shutil

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata, visualization

today = datetime.today().date()

### 4.1.2. Initialize Hopsworks Connection

In [ ]:
def detect_environment():
    if (
        "HOPSWORKS_JOB_ID" in os.environ
        or "HOPSWORKS_PROJECT_ID" in os.environ
        or "HOPSWORKS_JOB_NAME" in os.environ
    ):
        return "job"

    cwd = os.getcwd()
    if cwd.startswith("/hopsfs/Jupyter"):
        return "jupyter"

    return "local"

env = detect_environment()
print(f"Detected environment: {env}")

if env in ("job", "jupyter"):
    project = hopsworks.login()
    secrets_api = hopsworks.get_secrets_api()

    for key in ["HOPSWORKS_API_KEY", "AQICN_API_KEY", "GH_PAT", "GH_USERNAME"]:
        os.environ[key] = secrets_api.get_secret(key).value

else:
    env_path = os.path.join(root_dir, '.env')
    load_dotenv(env_path)
    print(f"✅ Loaded .env from: {env_path}")

settings = config.HopsworksSettings()

HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

HOPSWORKS_ENDPOINT = os.environ.get('HOPSWORKS_ENDPOINT')
if HOPSWORKS_ENDPOINT:
    HOPSWORKS_HOST = HOPSWORKS_ENDPOINT.replace('https://', '').replace('http://', '')
    print(f"🔗 Using Hopsworks host: {HOPSWORKS_HOST}")
else:
    HOPSWORKS_HOST = None
    print("🔗 Using default Hopsworks host")

print(f"🔑 API key loaded: {HOPSWORKS_API_KEY[:20]}...{HOPSWORKS_API_KEY[-10:]}")

if HOPSWORKS_HOST:
    project = hopsworks.login(host=HOPSWORKS_HOST, api_key_value=HOPSWORKS_API_KEY)
else:
    project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)

fs = project.get_feature_store()

print("Environment initialized and Hopsworks connected!")
print(project.name)

### 4.1.3. Repository management

In [ ]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

### 4.1.4. Configure API Keys and Secrets

In [ ]:
secrets = hopsworks.get_secrets_api()

try:
    secrets.get_secret("AQICN_API_KEY")
except:
    secrets.create_secret("AQICN_API_KEY", settings.AQICN_API_KEY.get_secret_value())

### 4.1.5. Get Model Registry

In [ ]:
mr = project.get_model_registry()

## 4.2. Get Feature Groups and Sensor Locations

In [ ]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

aq_data = air_quality_fg.read()

if len(aq_data) == 0:
    print("⚠️ No air quality data found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

sensor_locations = metadata.get_sensor_locations_dict(air_quality_fg)
print(f"📍 Loaded locations for {len(sensor_locations)} sensors")

## 4.3. Load Data from Feature Store

### 4.3.1. Set Inference Dates

In [ ]:
past_date = today - timedelta(days=7)  # Get 7 days of historical data for feature engineering
future_date = today + timedelta(days=7)  # Get 7 days of future weather forecasts
today_short = today.strftime("%Y-%m-%d")

print(f"Inference period: {past_date} to {future_date}")
print(f"Today: {today_short}")

### 4.3.2. Load Weather Features

In [ ]:
max_retries = 3
for attempt in range(max_retries):
    try:
        batch_weather = weather_fg.filter(
            (weather_fg.date >= past_date) & (weather_fg.date <= future_date)
        ).read()
        break
    except Exception as e:
        if attempt < max_retries - 1:
            print(f"⚠️ Weather read attempt {attempt + 1} failed, retrying...")
            time.sleep(2 ** attempt)
        else:
            print(f"❌ Failed to read weather data after {max_retries} attempts")
            batch_weather = weather_fg.read()
            batch_weather = batch_weather[
                (batch_weather["date"] >= past_date) & (batch_weather["date"] <= future_date)
            ]

batch_weather["date"] = pd.to_datetime(batch_weather["date"]).dt.tz_localize(None)

print(f"Retrieved {len(batch_weather)} weather records from {past_date} to {future_date}")

### 4.3.3. Load Air Quality Features

In [ ]:
max_retries = 3
for attempt in range(max_retries):
    try:
        batch_airquality = air_quality_fg.filter(
            air_quality_fg.date >= past_date
        ).read()
        break
    except Exception as e:
        if attempt < max_retries - 1:
            print(f"⚠️ Air quality read attempt {attempt + 1} failed, retrying...")
            time.sleep(2 ** attempt)
        else:
            print(f"❌ Failed to read air quality data after {max_retries} attempts")
            batch_airquality = air_quality_fg.read()
            batch_airquality = batch_airquality[
                batch_airquality["date"] >= past_date
            ]

batch_airquality["date"] = pd.to_datetime(batch_airquality["date"]).dt.tz_localize(None)

print(f"Retrieved {len(batch_airquality)} air quality records from {past_date} to {today}")

## 4.4. Model Retrieval
Download trained XGBoost models from Hopsworks model registry for each sensor and extract feature names.

In [ ]:
MODEL_NAME_TEMPLATE = "air_quality_xgboost_model_{sensor_id}"

retrieved_models = {}

for sensor_id in sensor_locations.keys():
    model_name = MODEL_NAME_TEMPLATE.format(sensor_id=sensor_id)
    
    try:
        available_models = mr.get_models(name=model_name)
        if not available_models:
            print(f"⚠️ No model found for sensor {sensor_id}, skipping...")
            continue
        
        retrieved_model = max(available_models, key=lambda model: model.version)
        saved_model_dir = retrieved_model.download()
        
        xgb_model = XGBRegressor()
        xgb_model.load_model(saved_model_dir + "/model.json")
        booster = xgb_model.get_booster()
        
        retrieved_models[sensor_id] = retrieved_model, xgb_model, booster.feature_names
        
    except Exception as e:
        print(f"❌ Error loading model for sensor {sensor_id}: {e}")
        continue

print(f"✅ Retrieved {len(retrieved_models)} models from registry")
print(f"   Total sensors in feature store: {len(sensor_locations)}")

In [ ]:
print(f"Retrieved {len(retrieved_models)} models.")

## 4.5. Batch Prediction
Merge weather and air quality data, iteratively predict PM2.5 values for forecast days, update engineered features after each prediction, and store results

### 4.5.1. Batch Prediction Loop

In [ ]:
# Merge historical data with weather data
batch_data = pd.merge(batch_weather, batch_airquality, on=["date", "sensor_id"], how="left")
batch_data = batch_data.sort_values(["sensor_id", "date"])

In [ ]:
feature_cols = [
    "pm25_rolling_3d",
    "pm25_lag_1d",
    "pm25_lag_2d",
    "pm25_lag_3d",
    "pm25_nearby_avg",
]

In [ ]:
# Create a new columns, fill with NaN for now
batch_data["predicted_pm25"] = np.nan
batch_data["days_before_forecast_day"] = np.nan
for col in feature_cols:
    batch_data[f"predicted_{col}"] = np.nan
    

forecast_days = [pd.Timestamp(today) + pd.Timedelta(days=i) for i in range(7)]

# Ensure today is always included for UI display, even if we have some actual data
if today.strftime("%Y-%m-%d") not in forecast_days:
    forecast_days = np.append([pd.Timestamp(today)], forecast_days)
    print(f"ℹ️  Added today ({today}) to forecast_days for UI completeness")

In [ ]:
for target_day in forecast_days:
    # context with all sensors up to current day
    window = batch_data.loc[batch_data["date"] <= target_day].copy()
    day_rows = window[window["date"] == target_day]

    for _, row in day_rows.iterrows():
        sensor_id = row["sensor_id"]
        try:
            _, xgb_model, model_features = retrieved_models[sensor_id]
        except KeyError:
            print(f"No model for sensor {sensor_id}, skipping prediction for {target_day}.")
            continue
        features = (row.reindex(model_features).to_frame().T.apply(pd.to_numeric, errors="coerce"))
        y_hat = xgb_model.predict(features)[0]

        idx = batch_data.index[(batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)][0]
       
        if pd.isna(row["pm25"]):
            batch_data.at[idx, "pm25"] = y_hat
        batch_data.at[idx, "predicted_pm25"] = y_hat
        batch_data.at[idx, "days_before_forecast_day"] = (target_day - pd.Timestamp(today)).days

    # Recompute features for after filling this day
    temp_df = batch_data.loc[batch_data["date"] <= target_day].copy()
    temp_df = feature_engineering.add_rolling_window_feature(
        temp_df, window_days=3, column="pm25", new_column="pm25_rolling_3d"
    )
    temp_df = feature_engineering.add_lagged_features(temp_df, column="pm25", lags=[1, 2, 3])
    temp_df = feature_engineering.add_nearby_sensor_feature(
        temp_df,
        sensor_locations,
        column="pm25_lag_1d",
        n_closest=3,
        new_column="pm25_nearby_avg",
    )

    current_rows = temp_df[temp_df["date"] == target_day]
    for _, row in current_rows.iterrows():
        sensor_id = row["sensor_id"]
        mask = (batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)
        if mask.any():
            for col in feature_cols:
                batch_data.loc[mask, f"predicted_{col}"] = row[col]

predictions = batch_data.loc[
    batch_data["predicted_pm25"].notna(),
    ["date", "sensor_id", "predicted_pm25", "days_before_forecast_day"]
    + [f"predicted_{col}" for col in feature_cols],
].reset_index(drop=True)


### 4.5.2. Assemble Prediction Results

In [ ]:
predictions_df = predictions.copy()

print(f"✅ Generated {len(predictions_df)} prediction rows")
print(f"   Date range: {predictions_df['date'].min()} to {predictions_df['date'].max()}")
print(f"   Sensors: {predictions_df['sensor_id'].nunique()}")
print(f"   Forecast days: {sorted(predictions_df['days_before_forecast_day'].unique())}")

### 4.5.3. Save Predictions

In [ ]:
# Create or get predictions feature group (same as in training pipeline)
predictions_fg = fs.get_or_create_feature_group(
    name="aq_predictions",
    version=1,
    primary_key=["sensor_id", "date", "days_before_forecast_day"],
    description="Air Quality prediction monitoring",
    event_time="date"
)

# Insert predictions
if len(predictions) > 0:

    # Difference in types between feature stores
    if env in ("job", "jupyter"):
        predictions["sensor_id"] = predictions["sensor_id"].astype("int64")
    else:
       predictions["sensor_id"] = predictions["sensor_id"].astype("int32")

    max_retries = 5
    delay = 2  # seconds

    for attempt in range(1, max_retries + 1):
        try:
            predictions_fg.insert(predictions, write_options={"wait_for_job": False})
            print(f"✅ Insert successful on attempt {attempt}")
            break

        except Exception as e:
            print(f"⚠️ Insert failed on attempt {attempt}: {type(e).__name__}: {e}")

            if attempt == max_retries:
                print("❌ Max retries reached. Insert failed permanently.")
                raise

            sleep_time = delay * (2 ** (attempt - 1))
            print(f"⏳ Retrying in {sleep_time} seconds...")
            time.sleep(sleep_time)

    print(f"✅ Inserted {len(predictions)} predictions to {predictions_fg.name}")
else:
    print("⚠️ No predictions to insert")

## 4.6. Analysis & Visualization

### 4.6.1. Generate Forecast Plots

In [ ]:
dataset_api = project.get_dataset_api()
forecast_paths = []

for sensor_id, location in sensor_locations.items():
    sensor_forecast = predictions[predictions["sensor_id"] == sensor_id].copy()

    city, street = location["city"], location["street"]
    forecast_dir = Path(root_dir) / "frontend" / "sensor_images" / str(sensor_id)
    forecast_path = forecast_dir / f"{sensor_id}_{today_short}_forecast.png"
    forecast_dir.mkdir(parents=True, exist_ok=True)

    fig = visualization.plot_air_quality_forecast(
        location["city"],
        location["street"],
        sensor_forecast,
        str(forecast_path),
        hindcast=False,
    )
    plt.close(fig)
    forecast_paths.append((sensor_id, str(forecast_path)))

if not dataset_api.exists("Resources/airquality"):
    dataset_api.mkdir("Resources/airquality")

# Upload with retry logic and error handling
upload_success = 0
upload_failed = 0

for i, (sensor_id, forecast_path) in enumerate(forecast_paths):
    max_retries = 3
    retry_delay = 2  # seconds
    
    for attempt in range(max_retries):
        try:
            dataset_api.upload(
                forecast_path,
                f"Resources/airquality/{sensor_id}_{today_short}_forecast.png",
                overwrite=True,
            )
            upload_success += 1
            if (i + 1) % 20 == 0:  # Progress update every 20 uploads
                print(f"   Uploaded {i + 1}/{len(forecast_paths)} plots...")
            break  # Success, exit retry loop
            
        except (ConnectionError, ProtocolError, Timeout, RequestException) as e:
            if attempt < max_retries - 1:
                print(f"⚠️ Upload failed for sensor {sensor_id} (attempt {attempt + 1}/{max_retries}), retrying in {retry_delay}s...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                print(f"❌ Failed to upload for sensor {sensor_id} after {max_retries} attempts: {e}")
                upload_failed += 1
        except Exception as e:
            print(f"❌ Unexpected error uploading for sensor {sensor_id}: {e}")
            upload_failed += 1
            break
    
    # Small delay between uploads to avoid overwhelming the connection
    if i < len(forecast_paths) - 1:
        time.sleep(0.1)

print(f"✅ Upload complete: {upload_success} successful, {upload_failed} failed")
if upload_success > 0:
    print(f"   Forecast plots available in Hopsworks under {project.get_url()}/settings/fb/path/Resources/airquality")

### 4.6.2. Hindcast Analysis

In [ ]:
try:
    monitoring_df = predictions_fg.filter(predictions_fg.days_before_forecast_day == 1).read()
    monitoring_df["date"] = pd.to_datetime(monitoring_df["date"]).dt.tz_localize(None)
    print(f"✅ Successfully read {len(monitoring_df)} hindcast predictions")
except Exception as e:
    print(f"⚠️ Could not read monitoring data: {e}")
    print("Skipping hindcast analysis...")
    monitoring_df = pd.DataFrame()  # Empty dataframe to prevent further errors

if not monitoring_df.empty:
    air_quality_df = air_quality_fg.read()[["date", "sensor_id", "pm25"]]
    air_quality_df["date"] = pd.to_datetime(air_quality_df["date"]).dt.tz_localize(None)

    for sensor_id, location in sensor_locations.items():
        try:
            sensor_preds = monitoring_df[monitoring_df["sensor_id"] == sensor_id][["date", "predicted_pm25"]]
            
            if sensor_preds.empty:
                continue
                
            merged = sensor_preds.merge(
                air_quality_df[air_quality_df["sensor_id"] == sensor_id][["date", "pm25"]],
                on="date",
                how="inner",
            ).sort_values("date")

            city, street = location["city"], location["street"]
            hindcast_dir = Path(root_dir) / "frontend" / "sensor_images" / str(sensor_id)
            hindcast_path = hindcast_dir / f"{sensor_id}_{today_short}_hindcast.png"
            hindcast_dir.mkdir(parents=True, exist_ok=True)

            fig = visualization.plot_air_quality_forecast(
                city,
                street,
                merged if not merged.empty else sensor_preds.assign(pm25=np.nan),
                str(hindcast_path),
                hindcast=True,
            )
            if fig is not None:
                plt.close(fig)

            dataset_api.upload(
                str(hindcast_path),
                f"Resources/airquality/{sensor_id}_{today_short}_hindcast.png",
                overwrite=True,
            )

        except Exception as e:
            print(f"⚠️ Error processing hindcast for sensor {sensor_id}: {e}")

### 4.6.3. IDW Heatmap

#### 4.6.3.1. Load Grid Bounds

In [ ]:
print("\n🗺️ Loading grid bounds for interpolation...")

try:
    with open(f"{root_dir}/frontend/coordinates.json") as f:
        coords_data = json.load(f)
    grid_bounds = (
        float(coords_data["MIN_LONGITUDE"]),
        float(coords_data["MIN_LATITUDE"]),
        float(coords_data["MAX_LONGITUDE"]),
        float(coords_data["MAX_LATITUDE"])
    )

    print(f"   Loaded grid bounds from coordinates.json: {grid_bounds}")

except Exception as e:
    print(f"⚠️ Could not load coordinates.json: {e}")
    print("   Falling back to sensor-based bounds...")
    
    lons = [loc['longitude'] for loc in sensor_locations.values()]
    lats = [loc['latitude'] for loc in sensor_locations.values()]
    
    min_lon, max_lon = min(lons), max(lons)
    min_lat, max_lat = min(lats), max(lats)
    
    # Add 10% padding
    lon_padding = (max_lon - min_lon) * 0.1
    lat_padding = (max_lat - min_lat) * 0.1
    
    grid_bounds = (
        float(min_lon - lon_padding),
        float(min_lat - lat_padding),
        float(max_lon + lon_padding),
        float(max_lat + lat_padding)
    )
    
    print(f"   Calculated fallback grid bounds: {grid_bounds}")

#### 4.6.3.2. Prepare Interpolation DataFrame
Actual Day-0 + Predictions

In [ ]:
print("\n🧮 Preparing interpolation DataFrame...")

interpolation_df = predictions.copy()
interpolation_df["predicted_pm25"] = pd.to_numeric(interpolation_df["predicted_pm25"], errors='coerce')

# Get today's actual measurements
today_actual = batch_data[batch_data["date"] == today_short].copy()

if not today_actual.empty:
    today_actual = today_actual[["date", "sensor_id", "pm25"]].copy()
    today_actual["pm25"] = pd.to_numeric(today_actual["pm25"], errors='coerce')
    today_actual = today_actual[today_actual["pm25"].notna()]
    
    if len(today_actual) > 0:
        print(f"   Using {len(today_actual)} actual sensor measurements for day 0")
        # Remove predicted rows for today
        interpolation_df = interpolation_df[interpolation_df["date"] != pd.Timestamp(today)]
        # Add actual measurements for today
        interpolation_df = pd.concat([today_actual, interpolation_df], ignore_index=True)
    else:
        print("   ⚠️ No valid actual pm25 for today — using predictions")
        today_preds = interpolation_df[interpolation_df["date"] == pd.Timestamp(today)].copy()
        if not today_preds.empty:
            today_preds["pm25"] = today_preds["predicted_pm25"]
            interpolation_df = interpolation_df[interpolation_df["date"] != pd.Timestamp(today)]
            interpolation_df = pd.concat([today_preds, interpolation_df], ignore_index=True)

else:
    print("   ⚠️ No batch_data for today — using predictions")
    today_preds = interpolation_df[interpolation_df["date"] == pd.Timestamp(today)].copy()
    if not today_preds.empty:
        today_preds["pm25"] = today_preds["predicted_pm25"]
        interpolation_df = interpolation_df[interpolation_df["date"] != pd.Timestamp(today)]
        interpolation_df = pd.concat([today_preds, interpolation_df], ignore_index=True)

# Diagnostics
print("\n🔍 Interpolation DataFrame dtypes:")
print(f"   predicted_pm25: {interpolation_df['predicted_pm25'].dtype}")
if "pm25" in interpolation_df.columns:
    print(f"   pm25: {interpolation_df['pm25'].dtype}")
    print(f"   Sample pm25 values: {interpolation_df[interpolation_df['pm25'].notna()]['pm25'].head(3).tolist()}")

#### 4.6.3.3. Generate IDW Heatmap Images

In [ ]:
print("\n🗺️ Generating heatmap interpolation images...")

interpolation_dir = f"{root_dir}/models/interpolation"
frontend_interpolation_dir = f"{root_dir}/frontend/interpolation"

os.makedirs(interpolation_dir, exist_ok=True)
os.makedirs(frontend_interpolation_dir, exist_ok=True)

unique_dates = sorted(interpolation_df["date"].unique())

print(f"   Total forecast dates: {len(unique_dates)}")
print(f"   Today: {today}")
print(f"   Dates: {[str(d)[:10] for d in unique_dates]}")

successful_images = 0
failed_images = 0

for i, forecast_date in enumerate(unique_dates):
    forecast_date_short = forecast_date.strftime("%Y-%m-%d")
    days_ahead = (forecast_date - pd.Timestamp(today)).days

    output_png = f"{interpolation_dir}/forecast_interpolation_{days_ahead}d.png"
    frontend_png = f"{frontend_interpolation_dir}/forecast_interpolation_{days_ahead}d.png"

    try:
        visualization.plot_pm25_idw_heatmap(
            interpolation_df,
            sensor_locations,
            forecast_date,
            output_png,
            grid_bounds=grid_bounds,
            today=today,
        )

        shutil.copy2(output_png, frontend_png)

        if days_ahead == 0:
            from IPython.display import Image, display
            display(Image(filename=output_png))

        print("✅")
        successful_images += 1

    except Exception as e:
        print(f"❌ {type(e).__name__}: {str(e)[:100]}")
        failed_images += 1

print(f"\n📊 Heatmap generation complete: {successful_images} successful, {failed_images} failed")

if failed_images > 0:
    print(f"⚠️ Warning: {failed_images} images failed to generate")
if successful_images == 0:
    raise Exception("❌ All heatmap images failed — pipeline cannot complete")

## 4.7. Export Predictions for Frontend

### 4.7.1. Prepare Predictions

In [ ]:
print("\n📦 Preparing predictions for frontend export...")

# Debug: show available forecast days and prediction count before merging with metadata
print(f"\n🔍 Debug - Available days_before_forecast_day values: {sorted(predictions['days_before_forecast_day'].unique())}")
print(f"   Total predictions: {len(predictions)}")

frontend_predictions = predictions.copy()


# Build metadata dataframe safely
sensor_metadata_rows = []
for sensor_id, location_data in sensor_locations.items():
    sensor_metadata_rows.append({
        'sensor_id': int(sensor_id),  # force int
        'latitude': float(location_data['latitude']),
        'longitude': float(location_data['longitude']),
        'city': location_data.get('city', ''),
        'street': location_data.get('street', '')
    })

sensor_metadata_df = pd.DataFrame(sensor_metadata_rows)

# Ensure consistent dtypes
frontend_predictions['sensor_id'] = frontend_predictions['sensor_id'].astype(int)
sensor_metadata_df['sensor_id'] = sensor_metadata_df['sensor_id'].astype(int)

print(f"   Metadata sensors: {sensor_metadata_df['sensor_id'].nunique()}")
print(f"   Prediction sensors: {frontend_predictions['sensor_id'].nunique()}")

# Merge predictions with metadata
merged = frontend_predictions.merge(sensor_metadata_df, on='sensor_id', how='left')

# Validate coordinates
missing_coords = merged[merged['latitude'].isna() | merged['longitude'].isna()]
if not missing_coords.empty:
    print("\n❌ ERROR: Missing coordinates for these sensor IDs:")
    print(missing_coords['sensor_id'].unique())
    raise ValueError("Some prediction rows have no matching sensor metadata. Fix required.")

frontend_predictions = merged
print(f"   ✅ Coordinates added for {len(frontend_predictions)} rows")
print(frontend_predictions.head())

# Convert datetime to string for JSON
frontend_predictions["date"] = frontend_predictions["date"].astype(str)

### 4.7.2. Export Predictions JSON

In [ ]:
# Export predictions as JSON for frontend
print("\n📦 Exporting predictions for frontend...")

# Export to frontend directory
predictions_json_path = f"{root_dir}/frontend/predictions.json"
frontend_predictions.to_json(predictions_json_path, orient="records", indent=2)

print(f"✅ Exported {len(frontend_predictions)} predictions to frontend/predictions.json")
print(f"   Sensors: {frontend_predictions['sensor_id'].nunique()}")
if len(frontend_predictions) > 0:
    print(f"   Date range: {frontend_predictions['date'].min()} to {frontend_predictions['date'].max()}" if len(frontend_predictions) > 0 else "   No predictions")

## 4.8. Commit Frontend Artifacts to GitHub

In [ ]:
print("\n📤 Committing predictions and interpolation images to git...")

try:
    # List PNGs for debugging
    frontend_pngs = glob.glob(f"{root_dir}/frontend/interpolation/*.png")
    print(f"   Found {len(frontend_pngs)} PNGs in frontend/interpolation/")
    if frontend_pngs:
        print(f"   Example: {os.path.basename(frontend_pngs[0])}")

    # Configure git identity
    subprocess.run(["git", "config", "user.name", "Hopsworks Bot"], cwd=root_dir, check=True)
    subprocess.run(["git", "config", "user.email", "bot@hopsworks.ai"], cwd=root_dir, check=True)

    # Stage the generated artifacts
    print("   Staging frontend artifacts...")
    subprocess.run(["git", "add", "-f", "frontend/predictions.json"], cwd=root_dir, check=True)
    subprocess.run(["git", "add", "-f", "frontend/interpolation/"], cwd=root_dir, check=True)
    subprocess.run(["git", "add", "-f", "frontend/sensor_images/"], cwd=root_dir, check=True)

    # Check if anything changed
    status = subprocess.run(
        ["git", "status", "--porcelain"],
        cwd=root_dir,
        capture_output=True,
        text=True
    )

    if not status.stdout.strip():
        print("ℹ️  No changes to commit — predictions and heatmaps already up to date")

    else:
        print("   Changes detected:")
        print(status.stdout)

        # Commit ONLY the frontend artifacts
        commit = subprocess.run(
            [
                "git", "commit",
                "-m", f"Update predictions and heatmaps - {today_short}",
                "--", "frontend/predictions.json", "frontend/interpolation/", "frontend/sensor_images/"
            ],
            cwd=root_dir,
            capture_output=True,
            text=True
        )

        if commit.returncode == 0:
            print("✅ Commit successful")

            push = subprocess.run(
                ["git", "push"],
                cwd=root_dir,
                capture_output=True,
                text=True
            )

            if push.returncode == 0:
                print("✅ Successfully pushed updates to GitHub")
                print("   Netlify will rebuild automatically")
            else:
                print("❌ Git push failed")
                print(push.stderr)

        else:
            print("⚠️ Commit failed")
            print(commit.stderr)

except Exception as e:
    print(f"❌ Git operation error: {e}")
    print("   Files exported locally but not pushed to git")


### 4.9. Pipeline Completion

In [ ]:
print("=" * 80)
print("✅ BATCH INFERENCE PIPELINE COMPLETED SUCCESSFULLY")
print("=" * 80)
print(f"\n📊 Summary:")
print(f"   - Predictions generated: {len(predictions)}")
print(f"   - Heatmap images created: {len(interpolation_df['date'].unique())}")
print(f"   - Feature group: {predictions_fg.name} (v{predictions_fg.version})")
print("\n" + "=" * 80)